<h3>Amendments Log</h3>
<table style="width:100%">
  <thead>
    <tr>
      <th style="text-align:left">Version</th>
      <th style="text-align:left">Amended By</th>
      <th style="text-align:left">Date</th>
      <th style="text-align:left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1.0</td>
      <td>Gary Manley</td>
      <td>2025-12-05</td>
      <td>Initial Version: Metadata-driven View creation.</td>
    </tr>
  </tbody>
</table>

In [ ]:
# 1. SETUP & IMPORTS
import duckdb
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load Env
vLocalEnvPath = r"C:/Users/garym/Documents/GitHub/MovieReleases/.env"
if os.path.exists(vLocalEnvPath):
    load_dotenv(dotenv_path=vLocalEnvPath)
else:
    load_dotenv()

vMdToken = os.getenv("MOTHERDUCK_TOKEN")
if not vMdToken: raise RuntimeError("MOTHERDUCK_TOKEN missing")

# Connect
print("Connecting to MotherDuck...")
vCon = duckdb.connect(f"md:?motherduck_token={vMdToken}")

In [ ]:
# PARAMETERS / CONSTANTS
cNotebookName = "process_gold_views.ipynb"
vConfigPath = "config/gold_views.csv"

## 2. Load Configuration
Read the CSV file that defines the mapping rules.

In [ ]:
if os.path.exists(vConfigPath):
    dfConfig = pd.read_csv(vConfigPath)
    # Fill NaN filters with empty string
    dfConfig['filter_condition'] = dfConfig['filter_condition'].fillna('')
    print(f"Loaded {len(dfConfig)} view definitions.")
else:
    raise FileNotFoundError(f"Config file missing: {vConfigPath}")

## 3. Generate Views
Loop through the config and execute `CREATE OR REPLACE VIEW` statements.

In [ ]:
# Ensure Gold Schema Exists
vCon.sql("CREATE SCHEMA IF NOT EXISTS MovieReleases.gold")

for vIndex, vRow in dfConfig.iterrows():
    vViewName = vRow['target_view_name']
    vSchema = vRow['source_schema']
    vTable = vRow['source_table']
    vSrcColsRaw = vRow['source_columns']
    vTgtColsRaw = vRow['target_columns']
    vFilter = vRow['filter_condition']
    
    print(f"Processing View: {vViewName}...")
    
    # Parse pipe-separated lists
    vSrcList = vSrcColsRaw.split('|')
    vTgtList = vTgtColsRaw.split('|')
    
    # Validation: Counts must match
    if len(vSrcList) != len(vTgtList):
        print(f"  [ERROR] Column count mismatch for {vViewName}. Skipping.")
        continue
        
    # Build Selection List: src AS "Tgt"
    # We wrap the Target in quotes to handle spaces (e.g., "Movie Title")
    vSelectList = []
    for vS, vT in zip(vSrcList, vTgtList):
        # Clean up quotes from CSV if present, then add SQL quotes
        vCleanTgt = vT.replace('"', '').strip()
        vSelectList.append(f'{vS.strip()} AS "{vCleanTgt}"')
    
    vSelectClause = ",\n        ".join(vSelectList)
    vWhereClause = f"WHERE {vFilter}" if vFilter else ""
    
    # Build SQL
    vSql = f"""
        CREATE OR REPLACE VIEW MovieReleases.gold."{vViewName}" AS
        SELECT 
        {vSelectClause}
        FROM MovieReleases.{vSchema}.{vTable}
        {vWhereClause}
    """
    
    try:
        vCon.sql(vSql)
        print("  [SUCCESS] View created.")
    except Exception as e:
        print(f"  [FAILURE] {e}")

print("Gold Layer Processing Complete.")
vCon.close()